In [ ]:
## Este código implementa o algoritmo A-Estrela para resolver o problema do Quebra-Cabeça de Oito Peças

## Definição do funcionamento do jogo

In [ ]:
import numpy as np

In [ ]:
## Definição do tabuleiro
# initial_puzzle = np.asarray([[1,2,3],
#                              [4,None, 5],
#                              [6,7,8 ]])

# initial_puzzle = np.asarray([[1,8,3],
#                              [6,None,4],
#                              [7,2,5]])

initial_puzzle = np.asarray([
    [ 2,  5,  3,  4,  1],
    [ 6,  7,  8,  9, 10],
    [11, 12, 13, 14, 15],
    [16, 17, 18, 19, None],
    [21, 22, 23, 24, 20]
])




objective_puzzle = np.asarray([
    [ 1,  2,  3,  4,  5],
    [ 6,  7,  8,  9, 10],
    [11, 12, 13, 14, 15],
    [16, 17, 18, 19, 20],
    [21, 22, 23, 24, None]
])


## Define as regras de movimentações:
rule_action = {
    'top': (-1, 0),
    'bottom': (1, 0),
    'left': (0, -1),
    'right': (0, 1)
}


puzzle_shape = (5, 5)

if not puzzle_shape == initial_puzzle.shape:
    raise 'O tabuleiro inicial é obrigado a obedecer o puzzle_shape'

In [ ]:
def h1(puzzle):
    global objective_puzzle
    
    '''
        Esta heurística consiste em contabilizar quantas peças estão em locais errados.
        Esta é uma heurística simples.
    '''
    
    return (puzzle != objective_puzzle).sum()

In [ ]:
def h2(puzzle):
    global objective_puzzle
    
    '''
        Esta heurística consiste em calcular a distância de manhattan de cada peça até sua posição objetivo.
    '''
    distancias_manhattan = []
    
    for i in range(puzzle.shape[0]):
        for j in range(puzzle.shape[1]):
            position = np.where(objective_puzzle == puzzle[i][j])
            
            distancias_manhattan.append( int(abs(i - position[0]) + abs(j - position[1])) )
    
    return np.asarray(distancias_manhattan).sum()

In [ ]:
## Função de ação no tabuleiro
def expands_new_states(father_puzzle):
    global rule_action
    
    
    actions = []
    children_puzzles = []
    
    ## Itera todas as possíveis ações para expandir os novos estados.
    for act in rule_action.keys():
        ## Pega a posição do espaço vazio do tabuleiro
        position = np.where(father_puzzle == None)

        ## Realiza a ação de movimentar a peça vazia.
        new_position = (position[0] + rule_action[act][0], position[1] + rule_action[act][1])
        
        if action_is_valid(new_position):
            actions.append(act)
            child_puzzle = father_puzzle.copy()
            ## Atualiza o tabuleiro com o novo estado, após o movimento da posição vazia do tabuleiro.
            child_puzzle[position] = child_puzzle[new_position]
            child_puzzle[new_position] = None
            
            children_puzzles.append(child_puzzle)
    
    return (actions, children_puzzles)


## Função para avaliar quais são os possíveis movimentos.
def action_is_valid(new_position):
    global puzzle_shape
    max_height = puzzle_shape[0]
    max_width = puzzle_shape[1]
    
    if new_position[0] >= 0 and new_position[0] < max_height and new_position[1] >= 0 and new_position[1] < max_width:
        return True

In [ ]:
expands_new_states(initial_puzzle)

In [ ]:
def insert(arr, value):
    arr.append(value)
    i = len(arr) - 1
    while i != 0 and arr[(i-1)//2]['total_heuristic'] < arr[i]['total_heuristic']:
        arr[i], arr[(i-1)//2] = arr[(i-1)//2], arr[i]
        i = (i-1)//2

## Algoritmo A*

In [ ]:
def insert(arr, value):
    arr.append(value)
    i = len(arr) - 1
    while i != 0 and arr[(i-1)//2]['total_heuristic'] > arr[i]['total_heuristic']:
        arr[i], arr[(i-1)//2] = arr[(i-1)//2], arr[i]
        i = (i-1)//2

In [ ]:
def structure(action, actions, puzzle, heuristic, total_heuristic):
    struct = {}
    
    struct['action'] = action
    struct['actions'] = actions
    struct['puzzle'] = puzzle
    struct['heuristic'] = heuristic
    struct['total_heuristic'] = total_heuristic
    
    
    return struct

In [ ]:
moviments_optimal_soluction = []

In [ ]:
%%time

priority_queue = []

solucao_otima = {}

## Gera os primeiros filhos.
children = expands_new_states(initial_puzzle)
for action, child in zip(children[0], children[1]):
    new_structure = structure(action, [action], child, h2(child), h2(child))
    insert(priority_queue, new_structure)
    
    
executions = 0
while True:
    if executions >= 2000000:
        break
    child_expanding = priority_queue.pop(0)
    if child_expanding['heuristic'] == 0:
        print('Encontrou a solução ótima')
        solucao_otima = child_expanding
        break
    children = expands_new_states(child_expanding['puzzle'])
    for action, child in zip(children[0], children[1]):
        new_structure = structure(action, child_expanding['actions']+[action], child, h2(child), child_expanding['total_heuristic']+h2(child))
        insert(priority_queue, new_structure)
        
    
    
    executions+=1


In [ ]:
executions

In [ ]:
len(priority_queue)

In [ ]:
priority_queue

In [ ]:
solucao_otima

In [ ]:
import pygame
import numpy as np

# Inicialização do pygame
pygame.init()

# Definições
WIDTH, HEIGHT = 300, 300
CELL_SIZE = WIDTH // 3
WHITE = (255, 255, 255)
FONT_SIZE = 60

# Inicialização da tela e fonte
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("8-Puzzle")
font = pygame.font.SysFont(None, FONT_SIZE)

# Função para desenhar o tabuleiro
def draw_board(board):
    screen.fill(WHITE)
    for i in range(3):
        for j in range(3):
            if board[i][j]:
                text = font.render(str(board[i][j]), True, (0, 0, 0))
                pygame.draw.rect(screen, (150, 150, 150), (j*CELL_SIZE, i*CELL_SIZE, CELL_SIZE, CELL_SIZE))
                screen.blit(text, (j*CELL_SIZE + CELL_SIZE//3, i*CELL_SIZE))
    pygame.display.flip()

# Função para mover o espaço vazio
def move(board, direction):
    i, j = np.where(board == None)
    i, j = i[0], j[0]
    
    if direction == 'left' and j > 0:
        board[i][j], board[i][j-1] = board[i][j-1], board[i][j]
    elif direction == 'right' and j < 2:
        board[i][j], board[i][j+1] = board[i][j+1], board[i][j]
    elif direction == 'top' and i > 0:
        board[i][j], board[i-1][j] = board[i-1][j], board[i][j]
    elif direction == 'bottom' and i < 2:
        board[i][j], board[i+1][j] = board[i+1][j], board[i][j]

    # Animação simples: apenas redesenha o tabuleiro várias vezes enquanto move
    for _ in range(10):
        draw_board(board)
        pygame.time.wait(50)

# Tabuleiro inicial
board = initial_puzzle.copy()




for action in solucao_otima['actions']:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    draw_board(board)
    pygame.time.wait(1000)  # Espera 1 segundo para visualização

    # Aqui, você pode chamar a função move() para testar a animação
    # Por exemplo:
    move(board, action)
    pygame.time.wait(1000)

pygame.quit()

    

In [ ]:
import pygame
import numpy as np
import random

# Inicialização do pygame
pygame.init()


BOARD_SIZE = 5
# Definições
WIDTH, HEIGHT = 300, 300
CELL_SIZE = WIDTH // BOARD_SIZE
WHITE = (255, 255, 255)

# Inicialização da tela
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("8-Puzzle")

# Carregar a imagem
image = pygame.image.load('cachorro.jpg')
image = pygame.transform.scale(image, (WIDTH, HEIGHT))

# Função para desenhar o tabuleiro
def draw_board(board):
    screen.fill(WHITE)
    for i in range(BOARD_SIZE):
        for j in range(BOARD_SIZE):
            if board[i][j]:
                y, x = divmod(board[i][j]-1, BOARD_SIZE)
                screen.blit(image, (j*CELL_SIZE, i*CELL_SIZE), (x*CELL_SIZE, y*CELL_SIZE, CELL_SIZE, CELL_SIZE))
    pygame.display.flip()

# Função para mover o espaço vazio
def move(board, direction):
    i, j = np.where(board == None)
    i, j = i[0], j[0]
    
    if direction == 'left' and j > 0:
        board[i][j], board[i][j-1] = board[i][j-1], board[i][j]
    elif direction == 'right' and j < 2:
        board[i][j], board[i][j+1] = board[i][j+1], board[i][j]
    elif direction == 'top' and i > 0:
        board[i][j], board[i-1][j] = board[i-1][j], board[i][j]
    elif direction == 'bottom' and i < 2:
        board[i][j], board[i+1][j] = board[i+1][j], board[i][j]

    # Animação simples: apenas redesenha o tabuleiro várias vezes enquanto move
    for _ in range(10):
        draw_board(board)
        pygame.time.wait(50)


# Tabuleiro inicial
board = initial_puzzle.copy()


for action in solucao_otima['actions']:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    draw_board(board)
    pygame.time.wait(200)  # Espera 1 segundo para visualização

    # Aqui, você pode chamar a função move() para testar a animação
    # Por exemplo:
    move(board, action)
    pygame.time.wait(200)

pygame.quit()


In [ ]:
import random